# 2.  Data Cleansing


This is the ```second``` notebook of N-Part series of DSS challenge 2 notebooks.

- [Data Information](data_information)

- Data Cleaning 

- [Data Analysis](data_analysis.ipynb)


In this notebook, we will analyse the page content data. This is a continuation of the [Data Information]() notebook where we had a first look at the data and parsed the excel file to get data in csv format.

Here, we will do the data cleaning.

Let's start by loading the data.

In [1]:
import pandas as pd
import brainpostpy.brainpost_data_tidy as tidy

In [2]:
DATA_DIR = 'data'
content_data = 'content_data.csv'
page_view_data = 'page_view_data.csv' 

In [3]:
content_data_df = pd.read_csv(f'{DATA_DIR}/{content_data}')

In [4]:
content_data_df.head(2)

,Page,Source / Medium,Date Range,Pageviews,Unique Pageviews,Avg. Time on Page,Entrances,Bounce Rate,% Exit,Page Value
0,/weekly-brainpost/tag/sharp-wave+ripples,google / organic,"Oct 25, 2020 - Oct 31, 2020",1,1,00:00:00,1,1.0,1.0,0
1,/weekly-brainpost/tag/sharp-wave+ripples,google / organic,"Oct 18, 2020 - Oct 24, 2020",0,0,00:00:00,0,0.0,0.0,0


#### Understanding the data Columns

- Page
    - Page column contains the page information from the URL. The data here has plenty of information related to the page. We just need to organize it.
- Source/Medium
    - This is the source of the traffic. whether the traffic is organic or from Google or both.
- Date range
    - This is the date range for the information has start and end date separated by a hyphen.
- Pageviews
    - Count of page views
- Unique page Views
    - page views from unique user
- Average time spent on page
    - This is also self explanatory. The average time is given in the format HH:mm:ss
- Entrances
    - Number of entrances to the page
- Bounce Rate
    - Bounce rate of the page
- % Exit
    - Percentage of exit 
- Page Value
    - [Page value](https://support.google.com/analytics/answer/2695658?hl=en)
    
    

There is so much information, but we need to tidy up the data before we can see some useful insights.

### Data Cleanup
- Let's start cleaning up.  First of all let's first rename the columns.

In [5]:
content_data_df = tidy.tidy_column_names(content_data_df)
content_data_df.head(1)

,page,source_medium,date_range,pageviews,unique_pageviews,avg_time_on_page,entrances,bounce_rate,percent_exit,page_value
0,/weekly-brainpost/tag/sharp-wave+ripples,google / organic,"Oct 25, 2020 - Oct 31, 2020",1,1,00:00:00,1,1.0,1.0,0


- Let's keep the ```page``` column as it is for a while. There is a lot to do with that column. we can do it later.
- Second column is ```source_medium```, It is a good idea to split ```source``` and ```column```.
- We can do the same for ```date_range``` column. Date range can be better represented in two columns as ```start_date``` and ```end_date```.

In [6]:
#source_medium
content_data_df[['source','medium']] = content_data_df.source_medium.str.split("/",expand=True,)
content_data_df = content_data_df.drop(['source_medium'], axis=1)
#date_range
content_data_df[['start_date','end_date']] = content_data_df.date_range.str.split("-",expand=True,)
content_data_df = content_data_df.drop(['date_range'], axis=1)
content_data_df.head(1)
content_data_df.columns.str.strip() #remve space aroudn the words

Index(['page', 'pageviews', 'unique_pageviews', 'avg_time_on_page',
       'entrances', 'bounce_rate', 'percent_exit', 'page_value', 'source',
       'medium', 'start_date', 'end_date'],
      dtype='object')

- Now, the time in ```avg_time_on_page``` column is in hour:minute:seconds format. Let's make it numeric by converting to seconds.

In [7]:
content_data_df['avg_time_on_page'] = content_data_df['avg_time_on_page'].apply(tidy.time_hhmmss_to_sec)

Let's have a look at ```page_value``` column.
[Page Value](https://support.google.com/analytics/answer/2695658?hl=en) is the average value for a page that a user visited before landing on the goal page or completing an Ecommerce transaction (or both). 

In [8]:
content_data_df['page_value'].unique()

array([0])

 ```page_value``` column does not have any meaningful data, hence it's not worth keeping. Let's remove that column.

In [9]:
content_data_df = content_data_df.drop(['page_value'], axis=1)
content_data_df.head(1)

,page,pageviews,unique_pageviews,avg_time_on_page,entrances,bounce_rate,percent_exit,source,medium,start_date,end_date
0,/weekly-brainpost/tag/sharp-wave+ripples,1,1,0,1,1.0,1.0,google,organic,"Oct 25, 2020","Oct 31, 2020"


###### Now let's get some useful information from the ```page``` column
- As the ```page``` column gives us information about the page url, as the url is well optimized, it gives clear information about which category the page belongs. We can see some tag names too. So let's clean it up...
- We are interested in paths and querystrings let's check what kind of query strings we have.

In [10]:
page_vals = content_data_df['page'].values

In [11]:
tidy.check_query_keys(page_vals)

Not sure what all of these query strings means but some of them look interesting and useful:

- fbclid: Means traffic from link sharing on facebook.
- rq : when the page is opened from search result
- month:
- offset: pagination is clicked
- q : search on the site
- depth:
- back: back link from google.
- s: 
- platform:
- ss_source:
- amp:
- sqsscreenshot:
- donatePageId:
- url:

From, all these query strings we can obtain important information about traffic.
Let's extend our dataframe with few extra columns, just in case they can be handy later..

In [12]:
content_data_df = tidy.extend_data_with_info_from_page(content_data_df)
content_data_df.head()

,page,pageviews,unique_pageviews,avg_time_on_page,entrances,bounce_rate,percent_exit,source,medium,start_date,end_date,path,from_facebook,google_keyword,from_google,search_keyword,sqsscreenshot,platform
0,/weekly-brainpost/tag/sharp-wave+ripples,1,1,0,1,1.0,1.0,google,organic,"Oct 25, 2020","Oct 31, 2020",/weekly-brainpost/tag/sharp-wave+ripples,NaN,NaN,NaN,NaN,NaN,NaN
1,/weekly-brainpost/tag/sharp-wave+ripples,0,0,0,0,0.0,0.0,google,organic,"Oct 18, 2020","Oct 24, 2020",/weekly-brainpost/tag/sharp-wave+ripples,NaN,NaN,NaN,NaN,NaN,NaN
2,/weekly-brainpost/tag/precentral+gyrus,0,0,0,0,0.0,0.0,google,organic,"Oct 25, 2020","Oct 31, 2020",/weekly-brainpost/tag/precentral+gyrus,NaN,NaN,NaN,NaN,NaN,NaN
3,/weekly-brainpost/tag/precentral+gyrus,1,1,0,1,1.0,1.0,google,organic,"Oct 18, 2020","Oct 24, 2020",/weekly-brainpost/tag/precentral+gyrus,NaN,NaN,NaN,NaN,NaN,NaN
4,/weekly-brainpost/tag/NMDA+blockers,2,2,414,2,0.5,0.5,google,organic,"Oct 25, 2020","Oct 31, 2020",/weekly-brainpost/tag/NMDA+blockers,NaN,NaN,NaN,NaN,NaN,NaN


For future analysis, let's save the dataframe in new csv file.

In [16]:
content_data_df.to_csv(f'{DATA_DIR}/ext_df_for_analysis.csv', index=False)

Now, we have an extended dataframe, We can do analysis in next Notebook. - [Data Analysis](data_analysis.ipynb)